In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf
import pandas as pd
import re


gpt2.download_gpt2(model_name="355M")

gpt2.mount_gdrive()

gpt2.copy_checkpoint_from_gdrive(run_name='run2')

gpt2.copy_file_from_gdrive('gpt2_generate.csv')
df = pd.read_csv('gpt2_generate.csv')


     |████████████████████████████████| 675kB 13.8MB/s 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 352Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 130Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 514Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:09, 144Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 208Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 144Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 208Mit/s]                                                       


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
results = []
i = 800
while i < 1200:
  print(i)
  tf.reset_default_graph()
  with gpt2.start_tf_sess() as sess:
    gpt2.load_gpt2(sess, run_name='run2')
    j = 0
    while j < df.shape[0] and j < 5:
      comment = df.loc[i,'comment']
      value = comment + ' [SEP] '
      try:
        %time text = gpt2.generate(sess, seed=1, run_name='run2', return_as_list=True, prefix=value, truncate='<|endoftext|>', nsamples=10, batch_size=10, temperature = 0.7, top_k = 20)
        text_clean = [(comment, re.sub('\n+', '\n',re.sub("&amp;#x200B;", "", t[len(value):])).strip()) for t in text]
        results.extend(text_clean)
      except Exception as e:
        print('generation failed! skipping')
        print(e)
      i += 1
      j += 1
fake_out = pd.DataFrame(results, columns = ['comment','reply'])
fake_out['real']=0 
fake_out.to_csv(f'bert_gan_fake{i}.csv', index = False)
gpt2.copy_file_to_gdrive(f'bert_gan_fake{i}.csv')

800
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
CPU times: user 25.3 s, sys: 2 s, total: 27.3 s
Wall time: 44.5 s
CPU times: user 26.3 s, sys: 1.66 s, total: 28 s
Wall time: 44.7 s
CPU times: user 23.3 s, sys: 1.49 s, total: 24.8 s
Wall time: 41 s
CPU times: user 18.6 s, sys: 1.06 s, total: 19.6 s
Wall time: 33 s
CPU times: user 27.2 s, sys: 1.49 s, total: 28.7 s
Wall time: 45.3 s
805
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
CPU times: user 25 s, sys: 1.49 s, total: 26.5 s
Wall time: 43.1 s
CPU times: user 23 s, sys: 1.31 s, total: 24.4 s
Wall time: 40.3 s
CPU times: user 21.2 s, sys: 1.17 s, total: 22.4 s
Wall time: 37.5 s
CPU times: user 26.9 s, sys: 1.49 s, total: 28.4 s
Wall time: 44.9 s
CPU times: user 27.2 s, sys: 1.51 s, total: 28.7 s
Wall time

In [0]:
# fake_out = pd.DataFrame(results, columns = ['reply'])
# fake_out['real']=0 
# fake_out.to_csv(f'bert_gan_fake{i}.csv')
# gpt2.copy_file_to_gdrive(f'bert_gan_fake{i}.csv')